In [1]:
from aiida import load_profile, orm
## Load your profile here
load_profile('bandgap') 
from aiida.plugins import WorkflowFactory
from aiida.engine import submit

In [6]:
code = orm.load_code(label='pw-qe-6.5-sirius')
overrides = {'pseudo_family': 'SSSP/1.1.2/PBEsol/efficiency'}
## Use the bundled test structure here
structure = orm.load_node('4522933d-83d8-4aba-a3a2-425a48cce792')

submit_OCV_workchain(structure, code, overrides)

<WorkChainNode: uuid: 8a803a44-d813-4fc2-94e0-e2112fb5cfd6 (pk: 98291) (aiida.workflows:quantumespresso.ocv.ocvwc)>

In [5]:
## Submitting the OCVWorkChain
def submit_OCV_workchain(structure, code, overrides, discharged_unitcell_relaxed=None, charged_unitcell_relaxed=None, time=28800, num_machines=4, num_mpiprocs_per_machine=12, num_cores_per_mpiproc=1):

    OCVWorkChain = WorkflowFactory('quantumespresso.ocv.ocvwc')

    builder = OCVWorkChain.get_builder_from_protocol(code, structure=structure, overrides=overrides, discharged_unitcell_relaxed=discharged_unitcell_relaxed, charged_unitcell_relaxed=charged_unitcell_relaxed)    

    builder.update({'clean_workdir':orm.Bool(False)})

    builder.ocv_relax['base']['pw']['metadata']['options'].update({'max_wallclock_seconds': time})
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
    builder.ocv_relax['base']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc

    builder.ocv_relax['base_final_scf']['pw']['metadata']['options'].update({'max_wallclock_seconds': time})
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_machines'] = num_machines
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_mpiprocs_per_machine'] = num_mpiprocs_per_machine
    builder.ocv_relax['base_final_scf']['pw']['metadata']['options']['resources']['num_cores_per_mpiproc'] = num_cores_per_mpiproc
    builder.ocv_parameters['distance'] = 8
    # builder.ocv_parameters['discharged_energy'] = -3936.9862933936
    # builder.ocv_parameters['charged_energy'] = -3541.4952435868
    
    # LFPO specific parameters
    builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['smearing'] = 'gauss'
    builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['degauss'] = 0.005 
    builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['starting_magnetization(1)'] = 0.5
    builder.ocv_relax['base']['pw']['parameters']['SYSTEM']['starting_magnetization(2)'] = -0.5

    builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['diagonalization'] = 'david'
    builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['mixing_mode'] = 'local-TF'
    builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['mixing_beta'] = 0.1
    builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 500
    # Options to decrease precision manyfolds
    # builder.ocv_relax['base']['pw']['parameters']['ELECTRONS']['conv_thr'] = 5e-5
    # builder.ocv_relax['base']['pw']['parameters']['CONTROL']['etot_conv_thr'] = 5e-3
    # builder.ocv_relax['base']['pw']['parameters']['CONTROL']['forc_conv_thr'] = 1e-3

    builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['smearing'] = 'mv'
    builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['degauss'] = 0.005 
    builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['starting_magnetization(1)'] = 0.5
    builder.ocv_relax['base_final_scf']['pw']['parameters']['SYSTEM']['starting_magnetization(2)'] = -0.5

    builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['diagonalization'] = 'david'
    builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['mixing_mode'] = 'local-TF'
    builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['mixing_beta'] = 0.1
    builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['electron_maxstep'] = 700
    builder.ocv_relax['base_final_scf']['pw']['parameters']['ELECTRONS']['conv_thr'] = 5e-8
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['CONTROL']['etot_conv_thr'] = 5e-3
    # builder.ocv_relax['base_final_scf']['pw']['parameters']['CONTROL']['forc_conv_thr'] = 1e-3

    return submit(builder)